In [1]:
import sys
sys.path.append('..')
sys.path.append('../ehrshot')
import copy
from typing import Literal
import argparse
import pandas as pd
import numpy as np
import os

import torch
from torch import nn
from pathlib import Path

from torch_uncertainty.utils import TUTrainer
from torch import nn

from torch_uncertainty.datamodules import MNISTDataModule
from torch_uncertainty.models.lenet import lenet
from torch_uncertainty.models import mc_dropout
from torch_uncertainty.models.mlp import bayesian_mlp

from torch_uncertainty.optim_recipes import optim_cifar10_resnet18
from torch_uncertainty.routines import ClassificationRoutine

from pathlib import Path

from lightning.pytorch import Trainer
from torch import nn, optim
from torch_uncertainty.losses import ELBOLoss


from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score
from tqdm import tqdm

In [2]:
results_dict = {}

labeling_functions=[
    "guo_los",
    "guo_readmission",
    "guo_icu",
    "new_hypertension",
    "new_hyperlipidemia",
    "new_pancan",
    "new_celiac",
    "new_lupus",
    "new_acutemi",
    "lab_thrombocytopenia",
    "lab_hyperkalemia",
    "lab_hyponatremia",
    "lab_anemia",
    "lab_hypoglycemia" # will OOM at 200G on `gpu` partition
]

unique_tasks_1 = ['guo_los', 'guo_readmission', 'guo_icu']
unique_tasks_2 = ['new_hypertension', 'new_hyperlipidemia', 'new_pancan', 'new_celiac', 'new_lupus', 'new_acutemi']
unique_tasks_3 = ['lab_thrombocytopenia', 'lab_hyperkalemia', 'lab_hyponatremia', 'lab_anemia', 'lab_hypoglycemia']

all_tasks = [unique_tasks_1, unique_tasks_2, unique_tasks_3]
all_tasks_name = ['unique_tasks_1', 'unique_tasks_2', 'unique_tasks_3']

In [3]:
def optim_recipe(model, lr_mult: float = 1.0):
    optimizer = torch.optim.SGD(model.parameters(), lr=0.05 * lr_mult)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)
    return {"optimizer": optimizer, "scheduler": scheduler}

max_epochs = 50
batch_size = 64

for i in tqdm(range(len(all_tasks))):
# for i in [0]:

    general_task_name = all_tasks_name[i]

    folder_path = f'multi_task_data_uq/{general_task_name}'

    train_x_name = os.path.join(folder_path, 'X_train_all.csv')
    train_y_name = os.path.join(folder_path, 'y_train_all.csv')
    val_x_name = os.path.join(folder_path, 'X_val_all.csv')
    val_y_name = os.path.join(folder_path, 'y_val_all.csv')

    X_train = pd.read_csv(train_x_name).to_numpy()
    y_train = pd.read_csv(train_y_name).to_numpy().reshape(-1)
    X_val = pd.read_csv(val_x_name).to_numpy()
    y_val = pd.read_csv(val_y_name).to_numpy().reshape(-1)

    # Create class weights
    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weights = torch.tensor(class_weights, dtype=torch.float)

    X_train = torch.tensor(X_train).float()
    X_val = torch.tensor(X_val).float()

    y_train = torch.tensor(y_train).long()
    y_val = torch.tensor(y_val).long()

    # Create TensorDatasets
    train_dataset = TensorDataset(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)

    # Create DataLoaders
    train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_dl = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    input_size = X_train.shape[1]
    hidden_size = 128
    num_classes = 2

    model = bayesian_mlp(in_features = input_size, num_outputs = num_classes, hidden_dims = [hidden_size])

    loss = ELBOLoss(
        model=model,
        inner_loss=nn.CrossEntropyLoss(weight=class_weights),
        kl_weight=1 / 500,
        num_samples=3,
    )

    routine = ClassificationRoutine(
        model=model,
        num_classes=2,
        loss=loss,
        optim_recipe=optim_recipe(model, 2),
        is_ensemble=True
    )

    trainer = Trainer(accelerator="gpu", enable_progress_bar=True, max_epochs=max_epochs)

    trainer.fit(routine, train_dataloaders=train_dl, val_dataloaders=val_dl)

    current_all_tasks_list = all_tasks[i]
    for j in tqdm(range(len(current_all_tasks_list))):
        folder_path_test = f'multi_task_data_uq/{general_task_name}/{current_all_tasks_list[j]}'
        
        test_x_name = os.path.join(folder_path_test, 'X_test.csv')
        test_y_name = os.path.join(folder_path_test, 'y_test.csv')

        X_test = pd.read_csv(test_x_name).to_numpy()
        y_test = pd.read_csv(test_y_name).to_numpy().reshape(-1)

        X_test = torch.tensor(X_test).float()
        y_test = torch.tensor(y_test).long()

        test_dataset = TensorDataset(X_test, y_test)
        test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

        ens_perf = trainer.test(routine, dataloaders=[test_dl])
        
        results_dict[current_all_tasks_list[j]] = ens_perf

  0%|          | 0/3 [00:00<?, ?it/s]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A10G') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ubuntu/anaconda3/lib/python3.11/site-packages/lightning/pytorch/core/optimizer.py:377: Found unsupported keys in the optimizer configuration: {'scheduler'}

  | Name                | Type             | Params | Mode 
-----------------------------------------------------------------
0 | model               | StochasticModel  | 230 K  | train
1 | loss                | ELBOLoss         | 230 K  | train
2 | format_batch_fn     | Identity         |

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/ubuntu/anaconda3/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.
/home/ubuntu/anaconda3/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/cal/ECE        │    0.12231845408678055    │
│       test/cal/aECE       │    0.12248274683952332    │
│       test/cls/Acc        │    0.7058550715446472     │
│      test/cls/Brier       │    0.3374636471271515     │
│       test/cls/NLL        │    0.5038435459136963     │
│     test/cls/entropy      │    0.5373411774635315     │
│   test/ens_Disagreement   │    0.35985708236694336    │
│     test/ens_Entropy      │    0.46124500036239624    │
│        test/ens_MI        │    0.14026470482349396    │
│       test/sc/AUGRC       │    0.07870995253324509    │
│       test/sc/AURC        │    0.10852422565221786    │
│    test/sc/CovAt5Risk     │            nan            │
│    test/sc/RiskAt80Cov    │    0.20092855393886566    │
└───────────────────────────┴───────────────────────────┘

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/ubuntu/anaconda3/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=63` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       test/cal/ECE        │    0.40203383564949036    │
│       test/cal/aECE       │    0.40203264355659485    │
│       test/cls/Acc        │    0.19921843707561493    │
│      test/cls/Brier       │    0.6934962272644043     │
│       test/cls/NLL        │    0.8992927074432373     │
│     test/cls/entropy      │     0.660252571105957     │
│   test/ens_Disagreement   │    0.38620659708976746    │
│     test/ens_Entropy      │    0.4765929579734802     │
│        test/ens_MI        │    0.1420038789510727     │
│       test/sc/AUGRC       │    0.4420032501220703     │
│       test/sc/AURC        │    0.9161989688873291     │
│    test/sc/CovAt5Risk     │            nan            │
│    test/sc/RiskAt80Cov    │    0.8567522168159485     │
└───────────────────────────┴───────────────────────────┘

100%|██████████| 3/3 [6:37:10<00:00, 7943.54s/it] 


In [4]:
import json
with open('results_model_uq/results_multi_task_bnn.json', 'w') as f:
    json.dump(results_dict, f)
current_all_tasks_list[j]

'lab_hypoglycemia'